In [206]:
import numpy as np
import pandas as pd
import math
import plotly.express as px
import statistics
import itertools as it
from functions import mms_enspectra_simulation_functions
import pytplot

In [207]:
ensize = len(mms_enspectra_simulation_functions.MMS_ENERGY_BIN)
vsize = 682
vel_set = np.arange(vsize)+10
dist_set = np.arange(5,61)
dsize = len(dist_set)

In [208]:
result = pd.DataFrame(np.array(np.meshgrid(vel_set, dist_set)).T.reshape(-1,2),columns=["velocity","distance_Re"])

result['distance'] = result['distance_Re'].apply(mms_enspectra_simulation_functions.convert_Re_to_km)
result['energy'] = result['velocity'].apply(mms_enspectra_simulation_functions.calcualte_energy_from_velocity)
result['time'] = result['distance'] / result['velocity'] / 60.
result['energy_binned'] = mms_enspectra_simulation_functions.find_energy_bin_df(result['energy'])

In [209]:
fig = px.line(result, x="time", y="energy_binned", color='distance_Re', log_y=True)
fig.update_xaxes(dtick=5)
fig.show()

In [210]:
target_distances = [5, 20, 60]
for target_distance in target_distances:
    index = result['distance_Re'] == target_distance
    
    fig = px.scatter(result.loc[index,:], x="time", y="energy_binned", log_y=True
                     , title= str(target_distance)+' Re')
    fig.update_xaxes(dtick=5)
    fig.show()

In [211]:
target_distances = [5,20,60]
avg_time = 5. 

for target_distance in target_distances:
    index = (result['distance_Re'] == target_distance)
    ntime = round(max(result.loc[index,'time'])/avg_time)
    time_loop = np.arange(0,ntime,1)
    energy_spectra = pd.DataFrame(data = np.empty((ntime,3)), columns =['time','energy','flux'])

    for itime in time_loop:
        st = itime * avg_time
        et = (itime+1) * avg_time
        index = (result['time'] >= st) & (result['time'] < et) & (result['distance_Re'] == target_distance)
        energy_spectra.loc[itime,'flux'] = len(result.loc[index,'energy_binned'])
        if energy_spectra.loc[itime,'flux'] > 0:
            energy_spectra.loc[itime,'energy'] = statistics.mean(result.loc[index,'energy_binned'])
        energy_spectra.loc[itime,'time'] = (st + et)/2
    
    energy_spectra['energy_binned'] = mms_enspectra_simulation_functions.find_energy_bin_df(
        energy_spectra['energy'])

    fig = px.scatter(energy_spectra, x="time", y="energy_binned", color = 'flux', log_y=True
                     , title = 'distance: ' + str(target_distance) + ' Re, ' 
                     + '   average time: ' + str(avg_time) + ' min')
    fig.update_xaxes(dtick=avg_time)
    fig.show()


In [212]:
target_distances = [5]
avg_time = 2. 

for target_distance in target_distances:
    index = (result['distance_Re'] == target_distance)
    ntime = round(max(result.loc[index,'time'])/avg_time)
    time_loop = np.arange(0,ntime,1)
    energy_spectra = pd.DataFrame(np.array(
        np.meshgrid(time_loop, mms_enspectra_simulation_functions.MMS_ENERGY_BIN)).T.reshape(-1,2)
                                  , columns=["itime","energy_binned"])
    
    energy_spectra['time'] = (energy_spectra['itime']+0.5)*avg_time
    energy_spectra['flux'] = energy_spectra.apply(
        lambda x: calculate_flux(x['itime'], x['energy_binned'], result, target_distance, avg_time), axis=1)      
index_valid = (energy_spectra['flux'] > 0)

beam =  pd.DataFrame(np.empty((ntime,2)), columns=['time','energy_binned'])
i = 0
for itime in np.unique(energy_spectra['time']):
    index = energy_spectra['time'] == itime
    beam.loc[i,'time'] = itime
    beam.loc[i, 'energy_binned'] = energy_spectra.loc[(energy_spectra.loc[index,'flux']).idxmax(),'energy_binned']
    i = i + 1
    
fig = px.scatter(energy_spectra.loc[index_valid,:], x="time", y="energy_binned", color = 'flux', log_y=True
                 , title = 'distance: ' + str(target_distance) + ' Re, ' 
                 + '   average time: ' + str(avg_time) + ' min')

reference_line = go.Scatter(x=list(beam['time']), y= list(beam['energy_binned'])
                            , mode="lines", line=go.scatter.Line(color="gray"), showlegend=False)

fig.add_trace(reference_line)

fig.update_xaxes(dtick=avg_time)
fig.show()


In [215]:
energy_spectra.loc[(energy_spectra.loc[index,'flux']).idxmax(),'energy_binned']

6.77

In [79]:
import plotly.graph_objects as go
import datetime
np.random.seed(1)

programmers = ['Alex','Nicole','Sara','Etienne','Chelsea','Jody','Marianne']

base = datetime.datetime.today()
dates = base - np.arange(180) * datetime.timedelta(days=1)
z = np.random.poisson(size=(len(programmers), len(dates)))

fig = go.Figure(data=go.Heatmap(z=z,x=dates,y=programmers,colorscale='Viridis'))

fig.update_layout(title='GitHub commits per day', xaxis_nticks=36)
z

array([[1, 0, 0, ..., 2, 0, 0],
       [0, 0, 1, ..., 2, 1, 1],
       [2, 2, 1, ..., 2, 0, 1],
       ...,
       [2, 0, 1, ..., 0, 2, 1],
       [1, 1, 2, ..., 2, 1, 1],
       [1, 1, 1, ..., 0, 0, 2]])

In [97]:
new_energy_spectra

time,2.5,7.5,12.5,17.5,22.5,27.5,32.5,37.5,42.5,47.5,52.5,57.5,62.5
energy_binned,,,,,,,,,,,,,
1.810000,0,0,0,0,0,0,0,0,0,0,0,0,0
3.510000,0,0,0,0,0,0,0,0,0,0,0,0,0
6.770000,0,0,0,0,0,0,0,0,0,0,0,0,1
13.120000,0,0,0,0,0,0,0,0,0,2,1,1,0
25.410000,0,0,0,0,0,0,2,3,1,0,0,0,0
49.200001,0,0,0,0,3,4,1,0,0,0,0,0,0
95.239998,0,0,0,8,3,0,0,0,0,0,0,0,0
184.380000,0,0,12,3,0,0,0,0,0,0,0,0,0
356.970000,0,12,9,0,0,0,0,0,0,0,0,0,0


In [96]:
fig = px.imshow(np.array(new_energy_spectra)
                , x= [2.5, 7.5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5, 42.5, 47.5, 52.5,
              57.5, 62.5]
                , y = [1.8099999,      3.51,      6.77,     13.12,     25.41, 49.200001,
              95.239998,    184.38,    356.97, 691.10999,   1338.04,   2590.49,
                5015.29,   9709.79,  18798.59,  32741.16]
                , title = 'distance: '+str(target_distance)+' Re, '+'   average time: '+str(avg_time)+' min')
#fig.update_xaxes(dtick=avg_time)
fig.show()

In [14]:
pytplot.store_data("enspectra"
                   , data={  'x': np.unique(energy_spectra['time'])
                           , 'y': np.unique(energy_spectra['flux']) 
                           , 'v': energy_spectra['energy_binned']})


The lengths of x and y do not match!


False